In [ ]:
import pandas as pd
import re

### MUDE O ATALHO DO CSV AQUI

In [ ]:
#muda o atalho no que ta entre aspas
atalho = r'PAC - Contatos NProd.csv'

#### Aqui pode mudar as colunas que deseja ver

In [ ]:
colunas = ['Semana', 'STATUS', 'Cargo', 'Portfólio', 'Hunter', 'Empresa']

In [ ]:
def sanitize_input(input_string):
    sanitized = re.sub(r'\s+', ' ', input_string).strip()
    sanitized = re.sub(r'[^\w\s]', '', sanitized)
    return sanitized

In [ ]:
def extract_date(semana_str):
    if isinstance(semana_str, str):
        match = re.search(r'\((\d{2}/\d{2}/\d{4})\)', semana_str)
        if match:
            return match.group(1)
    return None

In [ ]:
data_MMDD = pd.read_csv(
    atalho,
    usecols=colunas,
    low_memory=False,
    nrows=50795
)

In [ ]:
data_MMDD['Semana_date'] = data_MMDD['Semana'].apply(extract_date)
data_MMDD['Semana_mod'] = pd.to_datetime(data_MMDD['Semana_date'], format='%m/%d/%Y', errors='coerce')

In [ ]:
data_DDMM = pd.read_csv(
    atalho,
    usecols=colunas,
    low_memory=False,
    skiprows=range(1, 50796),
    header=0
)

In [ ]:
data_DDMM['Semana_date'] = data_DDMM['Semana'].apply(extract_date)
data_DDMM['Semana_mod'] = pd.to_datetime(data_DDMM['Semana_date'], format='%d/%m/%Y', errors='coerce')

In [ ]:
data = pd.concat([data_MMDD, data_DDMM], ignore_index=True)

In [ ]:
from datetime import datetime, timedelta
data_hoje = datetime.now()
data_maxima = data_hoje - timedelta(days=90)

In [ ]:
import re

def clean_string(s):
    if isinstance(s, str):
        return re.sub(r'[^a-zA-Z0-9\s&]', '', s).strip().lower()
    return ''

In [ ]:
data['Empresa'] = data['Empresa'].apply(clean_string)

eh so re-rodar essa celula de baixo, nao precisa rodar todo o codigo toda vez q for pesquisar

In [ ]:
valido = 1

nome = clean_string(input('Nome: ')).capitalize()

temp = data[pd.notna(data['Empresa']) & data['Empresa'].str.contains(nome, case=False, na=False)]

if not temp.empty:
    try:
        data_mod = temp['Semana_mod'].iloc[-1]
        data_original = temp['Semana'].iloc[-1]
    except:
        pass
    
    if any(temp['STATUS'].str.contains('projeto', case=False, na=False)):
        print(f"{nome} TEM PROJETO")
    if any(temp['STATUS'].str.contains('lead', case=False, na=False)):
        print(f"{nome} TEM LEAD")
    if data_mod > data_maxima:
        print(f"{nome} foi PROSPECTADO RECENTEMENTE: {data_original}")
        valido = 0
    temp = temp.drop(columns=['Semana_mod'])
    print(f"{nome}:")
    if(valido):
        print(f"Última vez prospectado em: {data_original}")
    print(temp.to_string(index=False))
else:
    print(f"{nome} nao encontrado")